NEd preds'  structure changed through pivoting the table. issue of slighlty deviating fetch_datetime stamps was dealt with.
NaN values occur for Type 59. I deleted that from dataset, for now..



In [9]:
import pandas as pd
import sqlite3
import pandas as pd


# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Read the raw_NED_preds table into a DataFrame
df_NED_preds_processed = pd.read_sql_query("SELECT * FROM raw_NED_preds", conn)

# Close the database connection
conn.close()


import pandas as pd

# Zet datumkolommen om naar datetime
df_NED_preds_processed['validfrom'] = pd.to_datetime(df_NED_preds_processed['validfrom'])
df_NED_preds_processed['validto'] = pd.to_datetime(df_NED_preds_processed['validto'])
df_NED_preds_processed['lastupdate'] = pd.to_datetime(df_NED_preds_processed['lastupdate'])
df_NED_preds_processed['current_datetime'] = pd.to_datetime(df_NED_preds_processed['current_datetime'])

# Zet 'lastupdate' als index en zorg dat deze ook datetime is
#df_NED_preds_processed.set_index('lastupdate', inplace=True)
#df_NED_preds_processed.index = pd.to_datetime(df_NED_preds_processed.index)



# Calculate rows where validfrom is earlier than lastupdate
mask = df_NED_preds_processed['validfrom'] >= df_NED_preds_processed['lastupdate']
# Keep only rows that match this condition, thereby dropping the rest
df_NED_preds_processed = df_NED_preds_processed[mask]
print(f"Number of rows where validfrom is earlier than lastupdate: {(~mask).sum()}")
# Sort by current_datetime to ensure correct grouping
df_NED_preds_processed = df_NED_preds_processed.sort_values('current_datetime')


# dealing with fact that current_datetime (the moment of fetching the data) is not always the same across NED Types fetched 'simultaneously':
# Group current_datetime values within 1 minute and assign the most recent (max) value in each group as 'fetch_moment':
df_NED_preds_processed['fetch_moment'] = (
    df_NED_preds_processed['current_datetime']
    .diff().gt(pd.Timedelta('1min')).cumsum()
)
# Map each group to its max current_datetime
fetch_moment_map = df_NED_preds_processed.groupby('fetch_moment')['current_datetime'].transform('max')
fetch_moment_map = pd.to_datetime(fetch_moment_map)
df_NED_preds_processed['fetch_moment'] = fetch_moment_map


# Verwijder irrelevante kolommen
df_NED_preds_processed = df_NED_preds_processed.drop(columns=[
    '@id', '@type', 'id', 'point', 'granularity', 'granularitytimezone', 'activity', 'classification', 'capacity','percentage','emission','emissionfactor','validfrom','lastupdate'])


# Verander de naam van de kolom 'type' naar 'PRED_Type' for clarity
df_NED_preds_processed['type'] = df_NED_preds_processed['type'].str.replace('/v1/types/', 'PRED_')
print("Unique values in 'type':", df_NED_preds_processed['type'].unique())

# Sort the DataFrame by the index,  by 'validto', asnd by 'type'

df_NED_preds_processed = df_NED_preds_processed.sort_values(by=['fetch_moment','validto', 'type'])
print(df_NED_preds_processed.head(20))



Number of rows where validfrom is earlier than lastupdate: 9251
Unique values in 'type': ['PRED_1' 'PRED_2' 'PRED_17' 'PRED_20' 'PRED_26' 'PRED_21' 'PRED_59']
          type    volume                   validto    current_datetime  \
1463    PRED_1   2141122 2025-04-02 09:00:00+00:00 2025-04-02 08:17:40   
29823  PRED_17   2799828 2025-04-02 09:00:00+00:00 2025-04-02 08:17:41   
30373   PRED_2  11265053 2025-04-02 09:00:00+00:00 2025-04-02 08:17:41   
1997   PRED_20    485000 2025-04-02 09:00:00+00:00 2025-04-02 08:17:42   
11076  PRED_21    365000 2025-04-02 09:00:00+00:00 2025-04-02 08:17:43   
36932  PRED_26    221000 2025-04-02 09:00:00+00:00 2025-04-02 08:17:43   
1464    PRED_1   2219175 2025-04-02 10:00:00+00:00 2025-04-02 08:17:40   
29824  PRED_17   2716766 2025-04-02 10:00:00+00:00 2025-04-02 08:17:41   
30374   PRED_2  14442681 2025-04-02 10:00:00+00:00 2025-04-02 08:17:41   
1998   PRED_20    485000 2025-04-02 10:00:00+00:00 2025-04-02 08:17:42   
11077  PRED_21    365000 20

In [10]:
# Pivot the table
pivoted_NED_preds = df_NED_preds_processed.pivot_table(
    index=['fetch_moment', 'validto'],  # keep these as index
    columns='type',                   # columns become unique values from 'type'
    values='volume',                  # values to fill in the new columns
    aggfunc='first'                   # if duplicates exist, take the first
)

# Optional: flatten column names and rename to match desired output
pivoted_NED_preds.columns = [f'Volume_{col}' for col in pivoted_NED_preds.columns]


# To avoid tuple-as-single-column issue:
pivoted_NED_preds = pivoted_NED_preds.reset_index()

# Remove the 'Volume_PRED_59' column from the pivoted DataFrame
pivoted_NED_preds = pivoted_NED_preds.drop(columns=['Volume_PRED_59'])
# Remove the 'Volume_PRED_21' and 'Volume_PRED_26' columns from the pivoted DataFrame, as theire values are constant
# and do not provide additional information
pivoted_NED_preds = pivoted_NED_preds.drop(columns=['Volume_PRED_21', 'Volume_PRED_26'])

print(pivoted_NED_preds.head())
print("Number of records in pivoted_NED_preds:", len(pivoted_NED_preds))


         fetch_moment                   validto  Volume_PRED_1  \
0 2025-04-02 08:17:43 2025-04-02 09:00:00+00:00      2141122.0   
1 2025-04-02 08:17:43 2025-04-02 10:00:00+00:00      2219175.0   
2 2025-04-02 08:17:43 2025-04-02 11:00:00+00:00      2379837.0   
3 2025-04-02 08:17:43 2025-04-02 12:00:00+00:00      2802929.0   
4 2025-04-02 08:17:43 2025-04-02 13:00:00+00:00      3373530.0   

   Volume_PRED_17  Volume_PRED_2  Volume_PRED_20  
0       2799828.0     11265053.0        485000.0  
1       2716766.0     14442681.0        485000.0  
2       2775487.0     17357228.0        485000.0  
3       2580169.0     18601126.0        485000.0  
4       2731686.0     18169871.0        485000.0  
Number of records in pivoted_NED_preds: 22161


In [11]:


# Print rows with missing values in any of the columns
missing_rows = pivoted_NED_preds[pivoted_NED_preds.isnull().any(axis=1)]
# Remove rows with missing values from the pivoted DataFrame
pivoted_NED_preds = pivoted_NED_preds.dropna()
print("Number of records in pivoted_NED_preds:", len(pivoted_NED_preds))

Number of records in pivoted_NED_preds: 22078


In [12]:
# checking for completeness of dataset
# Extract only date part from fetch_moment
fetch_dates = pivoted_NED_preds['fetch_moment'].dt.date

# Create full range of dates between min and max
date_range = pd.date_range(
    start=fetch_dates.min(),
    end=fetch_dates.max(),
    freq='D'
).date  # convert to plain date

# Find missing days
missing_dates = [d for d in date_range if d not in set(fetch_dates)]

# Print results

if not missing_dates:
    print("No missing dates in fetch_moment.")
else:
    print("Missing dates in fetch_moment:", missing_dates)

No missing dates in fetch_moment.


In [13]:
# ✅ Typecasting van datumkolommen
pivoted_NED_preds["validto"] = pd.to_datetime(pivoted_NED_preds["validto"], utc=True)
pivoted_NED_preds["fetch_moment"] = pd.to_datetime(pivoted_NED_preds["fetch_moment"], utc=True)

# ✅ Kolomnamen hernoemen
rename_map = {
    "Volume_PRED_1": "Wind_Vol",
    "Volume_PRED_17": "WindOffshore_Vol",
    "Volume_PRED_2": "Solar_Vol",
    "Volume_PRED_20": "Nuclear_Vol"
}
pivoted_NED_preds = pivoted_NED_preds.rename(columns=rename_map)

# ✅ Preview check
print(pivoted_NED_preds.dtypes)
print(pivoted_NED_preds.head())

fetch_moment        datetime64[ns, UTC]
validto             datetime64[ns, UTC]
Wind_Vol                        float64
WindOffshore_Vol                float64
Solar_Vol                       float64
Nuclear_Vol                     float64
dtype: object
               fetch_moment                   validto   Wind_Vol  \
0 2025-04-02 08:17:43+00:00 2025-04-02 09:00:00+00:00  2141122.0   
1 2025-04-02 08:17:43+00:00 2025-04-02 10:00:00+00:00  2219175.0   
2 2025-04-02 08:17:43+00:00 2025-04-02 11:00:00+00:00  2379837.0   
3 2025-04-02 08:17:43+00:00 2025-04-02 12:00:00+00:00  2802929.0   
4 2025-04-02 08:17:43+00:00 2025-04-02 13:00:00+00:00  3373530.0   

   WindOffshore_Vol   Solar_Vol  Nuclear_Vol  
0         2799828.0  11265053.0     485000.0  
1         2716766.0  14442681.0     485000.0  
2         2775487.0  17357228.0     485000.0  
3         2580169.0  18601126.0     485000.0  
4         2731686.0  18169871.0     485000.0  


In [14]:
# Save the processed DataFrame to a new SQLite database
output_db_path = '../data/WARP.db'
conn = sqlite3.connect(output_db_path)
pivoted_NED_preds.to_sql('processed_NED_preds', conn, if_exists='replace', index=True)
conn.close()